<div align="center"><h1> Projet HMIN230 </h1></div>
<div align="center"><h2> automatique de la langue naturelle </h2></div>
<h2>Membres du groupe</h2>
<ul>
    <li>Meriem AMERAOUI</li>
    <li>Dounia BELABIOD</li>
    <li>Bahaa Eddine NIL</li>
</ul>

<div class="alert alert-block alert-info" align="center">
    <h1>
        Imports
    </h1>
</div>

In [1]:
import re
import requests
import io
import itertools
import random

from IPython.display import clear_output
from bs4 import BeautifulSoup

<div class="alert alert-block alert-info" align="center">
    <h1>
        Fonctions
    </h1>
</div>

In [2]:
# Enlever une certains caractères dans une chaine de caractère
def rchop(thestring, ending):
    if thestring.endswith(ending):
        return thestring[:-len(ending)]
    return thestring

# Fonction d'affichage
def print_rules(bla):
    for rule in bla:
        print(str(rule).replace(', ', '==>'))

<div class="alert alert-block alert-info" align="center">
    <h1>
        Génération des mots
    </h1>
</div>

In [3]:
# Déclarations des variables
nature = {'Verbe': 'V', 'Nom': 'N', 'Adjectif': 'ADJ', 'Nom propre': 'NP'}
rules = []
words_list1 = {}

# Lecture du mot
print(f'Entrer le mot : ')
testWord = input()
clear_output(wait = False)

# Lecture de la nature du mot
print(f'Veuillez choisir la nature du mot dans la liste ci-dessous : ')
for (key, value) in nature.items():
    print(f'\t{value} pour {key}')
natureWord = input()
clear_output(wait = False)
while natureWord not in nature.values():
    print(f'Veuillez choisir la nature du mot dans la liste ci-dessous : ')
    for (key, value) in nature.items():
        print(f'\t{value} pour {key}')
    natureWord = input()
    clear_output(wait = False)

# Ouverture du fichier
file = open("rules.txt", "r")

# Lecture du fichier & enregistrement des règles
if file.mode == 'r':
    lines = file.readlines()
    for line in lines:
        rules.append(line.rstrip("\n").split("==>"))

# Fermeture du fichier
file.close()

# Génération des mots et leur nature
for rule in rules:
    flag = False
    ruleG = rule[0].split(":")
    ruleD = rule[1].split(":")
    match = re.match(".*" + ruleG[1], testWord)
    if match and ruleG[0] == natureWord:
        if ruleG[1] != "":
            generatedWord = rchop(testWord, ruleG[1]) + ruleD[1]
        else:
            generatedWord = testWord + ruleD[1]
        words_list1.setdefault(generatedWord, [])
        if(len(words_list1[generatedWord]) != 0):
            for n in words_list1[generatedWord]:
                if(n == ruleD[0]):
                    flag = True
            if(not flag):
                words_list1[generatedWord].append(ruleD[0])
        else:
            words_list1[generatedWord].append(ruleD[0])

# Affichage des mots
if(words_list1):
    print(f'\nLes dérivations du mot : {testWord}')
    for n in words_list1.items():
        temp = str(n).replace(", [", " de nature ").replace("[", "").replace("]", "").replace("'", "").replace("(", "").replace(")", "")
        print(f'\t{temp}')
else:
    print('Aucun mot n\'est valide')

# Affichage des règles
#print_rules(rules)


Les dérivations du mot : manger
	mangage de nature N
	mangement de nature N
	mangeur de nature AGENT, N
	mangeuse de nature AGENT
	mangateur de nature AGENT
	mangoire de nature LIEU
	mangoir de nature LIEU
	mangatoire de nature LIEU
	mange de nature N, V
	manges de nature V
	mangons de nature V
	mangez de nature V
	mangent de nature V
	mangeons de nature V
	mangant de nature ADJ, N
	mangette de nature N
	mangable de nature N
	mangible de nature ADJ
	mangail de nature N
	mangaille de nature N
	mangation de nature N
	mangeoire de nature LIEU
	mangeriteur de nature N
	mangerateur de nature N


<div class="alert alert-block alert-info" align="center">
    <h1>
        Mots existants dans jeux de mots
    </h1>
</div>

In [14]:
words_list2 = {}

for word in words_list1:
    # Envoie de la requete à jdm
    url = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=" + str(word) + "&rel="
    
    # Reponse de la requete
    response = requests.get(url)
    
    # bs4
    soup = BeautifulSoup(response.text, features = 'lxml')
    
    # Chercher la balise "jdm-warning"
    mydivs = soup.findAll("div", {"class": "jdm-warning"})
    
    if(not mydivs): # if jdm-warning existe (mot invalide)
        words_list2.setdefault(word, [])
        if(len(words_list2[word]) != 0):
            for n in words_list2[word]:
                if(n == words_list1[word]):
                    flag = True
            if(not flag):
                words_list2[word].append(words_list1[word])
        else:
            words_list2[word].append(words_list1[word])

# Affichage des mots
if(words_list2):
    print('Les mots existants dans jeux de mots :')
    for n in words_list2.items():
        temp = str(n).replace(", [", " de nature ").replace("[", "").replace("]", "").replace("'", "").replace("(", "").replace(")", "")
        print(f'\t{temp}')
else:
    print('Aucun mot n\'est valide')

Les mots existants dans jeux de mots :
	mangement de nature N
	mangeur de nature AGENT, N
	mangeuse de nature AGENT
	mange de nature N, V
	manges de nature V
	mangez de nature V
	mangent de nature V
	mangeons de nature V
	mangeoire de nature LIEU


<div class="alert alert-block alert-info" align="center">
    <h1>
        Mots en relation avec le mot de base
    </h1>
</div>

In [15]:
words_list3 = {}

for word in words_list2:
    a1 = ''
    # Envoie de la requete à jdm
    url = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=" + str(testWord) + "&rel=0"
    
    # Reponse de la requete
    response = requests.get(url)

    # bs4
    soup = BeautifulSoup(response.text, features = 'lxml')

    # Chercher la balise "CODE"
    mycode = soup.find('code')

    s = io.StringIO(str(mycode))
    for line in s:
        line = rchop(line, '\n')
        
        matchTestWord = re.match('e;.*;\'' + word + '\';.*;.*', line)
        if(matchTestWord):
            lineTestWord = line
            lineTestWordTemp = lineTestWord.split(';')
            a1 = lineTestWordTemp[1]
    
        if(a1):
            words_list3.setdefault(word, [])
            if(len(words_list3[word]) != 0):
                for n in words_list3[word]:
                    if(n == words_list1[word]):
                        flag = True
                if(not flag):
                    words_list3[word].append(words_list1[word])
            else:
                words_list3[word].append(words_list1[word])

# Affichage des mots
if(words_list3):
    print(f'Les mots existants dans jeux de mots qui sont en relation (r0) avec le mot : {testWord} :')
    for n in words_list3.items():
        temp = str(n).replace(", [", " de nature ").replace("[", "").replace("]", "").replace("'", "").replace("(", "").replace(")", "")
        print(f'\t{temp}')
else:
    print('Aucun mot n\'est valide')

Les mots existants dans jeux de mots qui sont en relation (r0) avec le mot : manger :
	mangeur de nature AGENT, N
	mange de nature N, V
	manges de nature V
	mangez de nature V
	mangent de nature V
	mangeons de nature V


<div class="alert alert-block alert-info" align="center">
    <h1>
        Mots finaux
    </h1>
</div>

In [20]:
print(words_list2)

{'mangement': [['N']], 'mangeur': [['AGENT', 'N']], 'mangeuse': [['AGENT']], 'mange': [['N', 'V']], 'manges': [['V']], 'mangez': [['V']], 'mangent': [['V']], 'mangeons': [['V']], 'mangeoire': [['LIEU']]}


In [25]:
relation_map = {'AGENT': '13', 'LIEU' : '15', 'N': '4', 'V': '4', 'ADJ': '4', 'ADV': '4'}
type_map = {'N': 'Nom:', 'V' : 'Ver:', 'ADJ' : 'Adj:'}
words_list4 = {}

def blabla(word, line):
    matchTestWord = re.match('e.*;.*;\'' + word + '\';.*;.*', line)
    if(matchTestWord):
        lineTestWord = line

for f in words_list3:   
    for k in itertools.chain.from_iterable(words_list3[f]):
        print(k)
        a1 = ''
        a2 = ''
        # Envoie de la requete à jdm
        if(int(relation_map[k]) == 4):
            url = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=" + str(f) + "&rel=" + str(relation_map[k])

            # Reponse de la requete
            response = requests.get(url)

            # bs4
            soup = BeautifulSoup(response.text, features = 'lxml')
            
            # Chercher la balise "CODE"
            mycode = soup.find('code')

            s = io.StringIO(str(mycode))
            for line in s:
                line = rchop(line, '\n')
            
                matchTestWord = re.match('e;.*;\'' + f + '\';.*;.*', line)
                if(matchTestWord):
                    lineTestWord = line
                    lineTestWordTemp = lineTestWord.split(';')
                    a1 = lineTestWordTemp[1]

                matchType = re.match('e;.*;\'' + str(type_map[k]) + '\';.*;.*', line)
                if(matchType):
                    lineType = line
                    lineTypeTemp = lineType.split(';')
                    a2 = lineTypeTemp[1]

                if(a2):
                    matchRule = re.match('r;.*;' + a1 + ';' + a2 + ';4;.*', line)
                    if(matchRule):
                        lineRule = line
                        lineRuleTemp = line.split(';')
                        a3 = int(lineRuleTemp[5])
                        if(a3 > 0):
                            words_list4.setdefault(f, [])
                            if(len(words_list4[f]) != 0):
                                for n in words_list4[f]:
                                    if(n == k):
                                        flag = True
                                if(not flag):
                                    words_list4[f].append(k)
                            else:
                                words_list4[f].append(k)
        else:
            url = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=" + str(testWord) + "&rel=" + str(relation_map[k])

            # Reponse de la requete
            response = requests.get(url)

            # bs4
            soup = BeautifulSoup(response.text, features = 'lxml')

            # Chercher la balise "CODE"
            mycode = soup.find('code')

            s = io.StringIO(str(mycode))
            for line in s:
                line = rchop(line, '\n')

                matchTestWord = re.match('e;.*;\'' + testWord + '\';.*;.*', line)
                if(matchTestWord):
                    lineTestWord = line
                    lineTestWordTemp = lineTestWord.split(';')
                    a1 = lineTestWordTemp[1]

                matchWord = re.match('e;.*;\'' + f + '\';.*;.*', line)
                if(matchWord):
                    lineWord = line
                    lineWordTemp = lineWord.split(';')
                    a2 = lineWordTemp[1]

                if(int(relation_map[k]) == 13):
                    matchRule = re.match('r;.*;' + a1 + ';' + a2 + ';13;.*', line)
                    if(matchRule):
                        lineRule = line
                        lineRuleTemp = line.split(';')
                        a3 = int(lineRuleTemp[5])
                        if(a3 > 0):
                            words_list4.setdefault(f, [])
                            if(len(words_list4[f]) != 0):
                                for n in words_list4[f]:
                                    if(n == k):
                                        flag = True
                                if(not flag):
                                    words_list4[f].append(k)
                            else:
                                words_list4[f].append(k)

                if(int(relation_map[k]) == 15):
                    matchRule = re.match('r;.*;' + a1 + ';' + a2 + ';15;.*', line)
                    if(matchRule):
                        lineRule = line
                        lineRuleTemp = line.split(';')
                        a3 = int(lineRuleTemp[5])
                        if(a3 > 0):
                            words_list4.setdefault(f, [])
                            if(len(words_list4[f]) != 0):
                                for n in words_list4[f]:
                                    if(n == k):
                                        flag = True
                                if(not flag):
                                    words_list4[f].append(k)
                            else:
                                words_list4[f].append(k)

# Affichage des mots
if(words_list4):
    print('Les mots finaux sont :')
    for n in words_list4.items():
        temp = str(n).replace(", [", " de nature ").replace("[", "").replace("]", "").replace("'", "").replace("(", "").replace(")", "")
        print(f'\t{temp}')
else:
    print('Aucun mot n\'est valide')

AGENT
N
N
V
V
V
V
V
Les mots finaux sont :
	mangeur de nature N
	mange de nature V
	manges de nature V
	mangez de nature V
	mangent de nature V
	mangeons de nature V
